# Analyse Rotation Spectra - Image Overview Hours


## Load Libraries

In [ ]:
from andbro__querrySeismoData import __querrySeismoData
from andbro__savefig import __savefig

from obspy import UTCDateTime
from numpy import log10, zeros, pi, append, linspace, mean, median, array, where, transpose, shape, histogram, arange
from numpy import logspace, linspace, log, log10, isinf, ones, nan, count_nonzero, sqrt, isnan
from pandas import DataFrame, concat, Series, date_range, read_csv, read_pickle
from tqdm import tqdm_notebook
from pathlib import Path
from scipy.stats import median_absolute_deviation as mad
from scipy.signal import welch

import os
import pickle
import matplotlib.pyplot as plt

In [ ]:
if os.uname().nodename == "lighthouse":
    root_path = "/home/andbro/kilauea-data/"
elif os.uname().nodename == "kilauea":
    root_path = "/import/kilauea-data/"

In [ ]:
def __get_minimal_psd(psds):

    from numpy import nanmin, array, nonzero
    
    min_psd = zeros(psds.shape[1])
    
    for f in range(psds.shape[1]):
        a = psds[:,f]
        min_psd[f] = nanmin(a[nonzero(a)])
    
    return min_psd

In [ ]:
def __get_median_psd(psds):

    from numpy import median, zeros, isnan

    med_psd = zeros(psds.shape[1])

    for f in range(psds.shape[1]):
        a = psds[:,f]
        med_psd[f] = median(a[~isnan(a)])

    return med_psd

In [ ]:
def __get_mean_psd(psds):

    from numpy import mean, zeros, isnan

    mean_psd = zeros(psds.shape[1])

    for f in range(psds.shape[1]):
        a = psds[:,f]
        mean_psd[f] = mean(a[~isnan(a)])

    return mean_psd

In [ ]:
def __get_minimum_psd(psds):
            
    for i, psd in enumerate(psds):
        if i == 0: 
            lowest_value = psd.sum()
            idx = 0

        value = psd.sum() 
        
        if value < lowest_value and value != 0:
            lowest_value = value
            idx = i   
    
    return psds[idx]

In [ ]:
def __load_psds(file, config):

    ## get data to dataframe and transpose and reindex
    df = read_csv(file, index_col=False)
    df = df.transpose()
    df.reset_index(inplace=True)
    # df.dtypes
    # df.mean()

    ## set column names
    try:
        columns = pickle.load(open(f"{config['inpath']}{config['inname']}_columns.pick", 'rb'))
        df.columns = [column.replace("-","") for column in columns]
    except:
        columns = arange(0,df.shape[1]).astype(str)
        df.columns = columns
        print(" -> Failed to assign column names! Assigned numbers instead!")

    ## check for column dublicates
    if len(df.columns.unique()) != len(df.columns):
        print(f" -> removing {len(df.columns)-len(df.columns.unique())} column dublicate(s)!")
        df = df.loc[:,~df.columns.duplicated()]

    count=0
    dates_expected = date_range(config['date1'].date, config['date2'].date, periods=int((config['date2']-config['date1'])/86400)+1)
    for dex in dates_expected:
        dex=str(dex.isoformat()[:10]).replace("-","")
        if not dex in df.columns:
            count+=1
    print(f" -> missing {count} days")
        
    print(f" -> total of {df.shape[0]} psds")
        
#     ## convert to list
#     psds = []
#     for col in array(df.columns):
        
#         ## turn non-float series to float objects
#         df[col] = pd.to_numeric(df[col], errors = 'coerce')

#         ## add to psds list
#         psds.append(array(df[col]))
        
    return df

In [ ]:
def __get_array_from_dataframe(df):
    
    from pandas import to_numeric
    
    ## convert to list
    psds = []
    for col in array(df.columns):
        
        ## turn non-float series to float objects
        df[col] = to_numeric(df[col], errors = 'coerce')

        ## add to psds list
        psds.append(array(df[col]))

    return array(psds)

In [ ]:
# def __remove_noisy_psds(df, threshold_mean=1e-13):

#     from numpy import delete
    
#     l1 = len(df.columns)
#     for col in df.columns:
# #         print(col, type(col))
#         if df[col].astype(float).mean() > threshold_mean:
#             df = df.drop(columns=col)
#     l2 = len(df.columns)
#     print(f" -> removed {l1-l2} columns due to mean thresholds!")
#     print(f" -> {l2} psds remain")
    
#     return df

In [ ]:
def __get_percentiles(arr):
    
    from numpy import zeros, nanpercentile
    
    percentiles_lower = zeros(shape(arr)[1])
    percentiles_upper = zeros(shape(arr)[1])

    for kk in range(shape(arr)[1]):
        out = nanpercentile(arr[:, kk],  [2.5 ,97.5])
        percentiles_upper[kk] = out[1]
        percentiles_lower[kk] = out[0]
        
    return percentiles_lower, percentiles_upper

## Configurations

In [ ]:
## _________________________________________________________

name = "BSPF"
inname = "2023_BSPF_Z_3600"  ## "2022_BSPF_Z_3600"
subdir = "BSPF_2023_Z/"  ## "BSPF_2022_Z/"
threshold = 1e-13 ## 8e-14
period_limits = 1/80, 30  ## 1/50, 30
vmin, vmax = 1e-16, 1e-14

# name = "PFOIX"
# inname = "2023_PFOIX_Z_3600"
# subdir = "PFOIX_2023_Z/"
# threshold = 1e-12
# period_limits = 1/80, 100  ## 1/50, 30
# vmin, vmax = 1e-16, 1e-12

## _________________________________________________________


path = f"{root_path}BSPF/data/"

config = pickle.load(open(path+subdir+inname+"_config.pkl", 'rb'))

config['inname'] = inname
config['inpath'] = path+subdir
config['period_limits'] = period_limits
config['thres'] = threshold

config['outpath_figures'] = f"{root_path}BSPF/figures/"

config['rlnm_model_path'] = f"{root_path}LNM/data/MODELS/"

config['frequency_limits'] = [1/config['period_limits'][1], 1/config['period_limits'][0]]


In [ ]:
!ls /home/andbro/kilauea-data/BSPF/data

## Load as Arrays

In [ ]:
def __load_data_files(config, path):
    
    from tqdm.notebook import tqdm
    from numpy import array
    
    print(path)
    
    config['files'] = [file for file in os.listdir(path) if "hourly" in file]
    config['files'].sort()

    psds_all, times_nom, times = [], arange(0, 24*365, 1), []
    count, missing  = 0, 0
    
    for file in tqdm(config['files']):
        date = file.split("_")[-2]
        psds_hourly = read_pickle(path+file)

        for h in range(24):
            try:
                psds_all.append(psds_hourly[h])
                times.append(times_nom[count])
            except:
                missing += 1
            count += 1
            
    print(f" -> missing: {missing}")
    return array(psds_all), times

In [ ]:
ADR_Z, times_Z = __load_data_files(config, config['inpath'].replace("Z","Z"))

ff_Z = pickle.load(open(f"{config['inpath'].replace('Z','Z')}{config['inname'].replace('Z','Z')}_frequency_axis.pkl", 'rb'))
times_Z = pickle.load(open(f"{config['inpath'].replace('Z','Z')}{config['inname'].replace('Z','Z')}_times_axis.pkl", 'rb'))

len(ADR_Z), len(ff_Z), len(times_Z)

In [ ]:
ADR_N, times_N = __load_data_files(config, config['inpath'].replace("Z","N"))

ff_N = pickle.load(open(f"{config['inpath'].replace('Z','N')}{config['inname'].replace('Z','N')}_frequency_axis.pkl", 'rb'))
times_N = pickle.load(open(f"{config['inpath'].replace('Z','N')}{config['inname'].replace('Z','N')}_times_axis.pkl", 'rb'))

len(ADR_N), len(ff_N), len(times_N)

In [ ]:
ADR_E, times_E = __load_data_files(config, config['inpath'].replace("Z","E"))

ff_E = pickle.load(open(f"{config['inpath'].replace('Z','E')}{config['inname'].replace('Z','E')}_frequency_axis.pkl", 'rb'))
times_E = pickle.load(open(f"{config['inpath'].replace('Z','E')}{config['inname'].replace('Z','E')}_times_axis.pkl", 'rb'))


len(ADR_E), len(ff_E), len(times_E)

### Cut Frequency Range

In [ ]:
def __cut_frequencies_array(arr, freqs, fmin, fmax):

    ind = []
    for i, f in enumerate(freqs):
        if f >= fmin and f <= fmax:
            ind.append(i)

    ff = freqs[ind[0]:ind[-1]]
    pp = arr[:,ind[0]:ind[-1]]
    
    return pp, ff 

In [ ]:
config['frequency_limits']

In [ ]:
ADR_N, ff_N = __cut_frequencies_array(ADR_N, ff_N, config['frequency_limits'][0], config['frequency_limits'][1])
ADR_E, ff_E = __cut_frequencies_array(ADR_E, ff_E, config['frequency_limits'][0], config['frequency_limits'][1])
ADR_Z, ff_Z = __cut_frequencies_array(ADR_Z, ff_Z, config['frequency_limits'][0], config['frequency_limits'][1])

## Remove Noisy PSDs

In [ ]:
def __remove_noisy_psds(arr, times, threshold_mean=1e-16):

    from numpy import delete, shape, sort, array, nan
    
    l1 = shape(arr)[0]

    idx_to_remove = []
    for ii in range(shape(arr)[0]):
        
        ## appy upper threshold
        if arr[ii,:].mean() > threshold_mean:
            idx_to_remove.append(ii)
            
        ## apply default lowe threshold
        if arr[ii,:].mean() < 1e-26:
            idx_to_remove.append(ii)
            
    for jj in sort(array(idx_to_remove))[::-1]:
        
        ## option 1: delte rows
#         arr = delete(arr, jj, axis=0)
#         times = delete(times, jj, axis=0)

        ## option 2: replace with nan values
        arr[jj,:] = ones(len(arr[jj]))*nan
    
    l2 = shape(arr)[0]
    
    print(f" -> removed {l1-l2} rows due to mean thresholds!")
    print(f" -> {l2} psds remain")
    
    return arr, times

In [ ]:
ADR_N, times_N = __remove_noisy_psds(ADR_N, times_N, threshold_mean=config['thres'])
ADR_E, times_E = __remove_noisy_psds(ADR_E, times_E, threshold_mean=config['thres'])
ADR_Z, times_Z = __remove_noisy_psds(ADR_Z, times_Z, threshold_mean=config['thres'])

## Write Statistics to Files

In [ ]:
def __write_stats_mod(config, ff, arr, comp):
    
    out_df = DataFrame()

    out_df['frequencies'] = ff
    out_df['minimal'] = __get_minimal_psd(arr)
    out_df['median'] = __get_median_psd(arr)
    out_df['minimum'] = __get_minimum_psd(arr)
    out_df['mean'] = __get_mean_psd(arr)

    print(f" -> writing {config['inpath'].replace('Z',comp)}{config['inname'].replace('Z',comp)}_stats_mod.pkl")
    out_df.to_pickle(f"{config['inpath'].replace('Z',comp)}{config['inname'].replace('Z',comp)}_stats_mod.pkl")

In [ ]:
# __write_stats_mod(config, ff_N, ADR_N, "N")
# __write_stats_mod(config, ff_E, ADR_E, "E")
# __write_stats_mod(config, ff_Z, ADR_Z, "Z")

## Check dimensions

In [ ]:
def __check():
    if len(times_N) != shape(ADR_N)[0]:
        print("-> times N:", len(times_N), shape(ADR_N)[0])
    if len(ff_N) != shape(ADR_N)[1]:
        print("-> freqs N:", len(ff_N), shape(ADR_N)[0])
    
    if len(times_E) != shape(ADR_E)[0]:
        print("-> times E:", len(times_E), shape(ADR_E)[0])
    if len(ff_E) != shape(ADR_E)[1]:
        print("-> freqs E:", len(ff_E), shape(ADR_E)[0])
        
    if len(times_Z) != shape(ADR_Z)[0]:
        print("-> times Z:", len(times_Z), shape(ADR_Z)[0])
    if len(ff_Z) != shape(ADR_Z)[1]:
        print("-> freqs Z:", len(ff_Z), shape(ADR_Z)[0])
        
__check()

## Plotting

In [ ]:
def __makeplot_image(ff, psds, times, dates=None):
    
    from tqdm.notebook import tqdm
    from numpy import isnan, median, mean, std, array, zeros, nanmax, nanmin
    from scipy.stats import median_abs_deviation as mad
    from matplotlib import colors


    ##____________________________
        
    fig, axes = plt.subplots(1, 1, figsize=(12,8), sharey=False, sharex=False)

    plt.subplots_adjust(hspace=0.1)
    
    font = 14

    N = int(24*365)

    
    cmap = plt.cm.get_cmap('rainbow')
#     cmap.set_under(color='black')
#     cmap.set_over(color='white')
    
    
    max_psds = nanmax(psds)
    min_psds = nanmin(psds)

    im = axes.pcolormesh(times, 1/array(ff), psds.T, 
                            cmap=cmap,
                            vmax=max_psds,
                            vmin=min_psds,
                            norm=colors.LogNorm(),
                            )

    axes.tick_params(labelsize=font-2)


    axes.set_ylim(config['period_limits'][0], config['period_limits'][1])
    axes.set_yscale("log")

    ## set colorbar at bottom
    cbar = fig.colorbar(im, orientation='vertical', ax=axes.ravel().tolist(), aspect=30, pad=0.01)
    cbar.set_label(r"PSD (rad$^2$/s$^2$/$Hz$)", fontsize=font-2, labelpad=-53)

    axes.set_xlabel("Days of 2019", fontsize=font, labelpad=-1)

    new_ticks = [int(round(t/24,0)) for t in axes.get_xticks()]
    axes.set_xticklabels(new_ticks)
    
    ## panel labels
    axes.text(.01, .99, 'a)', ha='left', va='top', transform=axes.transAxes, fontsize=font+2)
    
    ## data labels
    array = name.split("_")[0]
      
    axes.set_ylabel(r"Periods (s)", fontsize=font)
    
    plt.show();    
    return fig

In [ ]:
# nn = 500
# fig = __makeplot_image(times_N[:nn], ff_N, ADR_N[:nn,:])

# fig = __makeplot_image(times_N, ff_N, ADR_N)

In [ ]:
def __makeplot_image_overview(ff, psds, times):
    
    from tqdm.notebook import tqdm
    from numpy import isnan, median, mean, std, array, zeros, nanmax, nanmin
    from scipy.stats import median_abs_deviation as mad
    from matplotlib import colors

    
    ## theoretical rlnm
#     rlnm = read_csv(config['rlnm_model_path']+"rlnm_theory.csv")

    ## convert frequencies to periods
    pp=[]
    for mm in range(len(ff)):
        ppp = zeros(len(ff[mm]))
        ppp = 1/ff[mm]
        pp.append(ppp)
        
    ##____________________________
    
    NN = 3
    
    fig, axes = plt.subplots(NN, 1, figsize=(12,12), sharey=False, sharex=True)

    plt.subplots_adjust(hspace=0.1)
    
    font = 14

    N = int(24*365)

    max_psds = max(nanmax(psds[0]), nanmax(psds[1]), nanmax(psds[2]))
    min_psds = min(nanmin(psds[0]), nanmin(psds[1]), nanmin(psds[2]))
    
    cmap = plt.cm.get_cmap('rainbow')
#     cmap.set_under(color='black')
#     cmap.set_over(color='white')
    cmap.set_bad(color='white')

    
    for j in tqdm(range(NN)):
#         vmin, vmax = min_psds, max_psds,
        vmin, vmax = 1e-16, 1e-14
    
        im = axes[j].pcolormesh(times[j], 1/ff[j], psds[j].T, 
                                cmap=cmap,

                                norm=colors.LogNorm(vmin, vmax),
                                shading="auto",
                                )
        
        axes[j].tick_params(labelsize=font-2)
        

        axes[j].set_ylim(config['period_limits'][0], config['period_limits'][1])
        axes[j].set_yscale("log")

    axes[NN-1].set_xlabel("Days of 2019", fontsize=font, labelpad=-1)

    new_ticks = [int(round(t/24,0)) for t in axes[NN-1].get_xticks()]
    axes[NN-1].set_xticklabels(new_ticks)
    
    ## panel labels
    axes[0].text(.01, .99, 'a)', ha='left', va='top', transform=axes[0].transAxes, fontsize=font+2)
    axes[1].text(.01, .99, 'b)', ha='left', va='top', transform=axes[1].transAxes, fontsize=font+2)
    axes[2].text(.01, .99, 'c)', ha='left', va='top', transform=axes[2].transAxes, fontsize=font+2)
    
    ## data labels
    array = name.split("_")[0]
    axes[0].text(.99, .99, f'{array} ADR $vertical$', ha='right', va='top', transform=axes[0].transAxes, fontsize=font)
    axes[1].text(.99, .99, f'{array} ADR $north$', ha='right', va='top', transform=axes[1].transAxes, fontsize=font)
    axes[2].text(.99, .99, f'{array} ADR $east$', ha='right', va='top', transform=axes[2].transAxes, fontsize=font)
    
    
    axes[0].set_ylabel(r"Periods (s)", fontsize=font)
    axes[1].set_ylabel(r"Periods (s)", fontsize=font)
    axes[2].set_ylabel(r"Periods (s)", fontsize=font)
    
    ## set colorbar at bottom
    cbar = fig.colorbar(im, orientation='vertical', ax=axes.ravel().tolist(), aspect=30, pad=0.01)
    cbar.set_label(r"PSD (rad$^2$/s$^2$/$Hz$)", fontsize=font-2, labelpad=-53)

    plt.show();    
    return fig

In [ ]:
# nn = 24*30
# fig = __makeplot_image_overview([ff_Z, ff_N, ff_E], 
#                                      [ADR_Z[:nn,:], ADR_N[:nn,:], ADR_E[:nn,:]], 
#                                      [times_Z[:nn], times_N[:nn], times_E[:nn]],
#                                     )

# fig = __makeplot_image_overview([ff_Z, ff_N, ff_E], 
#                                 [ADR_Z, ADR_N, ADR_E], 
#                                 [times_N, times_N, times_E], 
#                                 )

In [ ]:
# __savefig(fig, outpath=config['outpath_figures'], outname=f"{name}_20230401_20230615_psdimage", mode="png", dpi=300)

### Plot: Version 2

In [ ]:
def __makeplot_image_overview2(ff, psds, times, dates=None):
    
    from tqdm.notebook import tqdm
    from numpy import isnan, median, mean, std, array, zeros, nanmax, nanmin
    from scipy.stats import median_abs_deviation as mad
    from matplotlib import colors

    
    ## theoretical rlnm
    rlnm = read_csv(config['rlnm_model_path']+"rlnm_theory.csv")

    ## convert frequencies to periods
    pp=[]
    for mm in range(len(ff)):
        ppp = zeros(len(ff[mm]))
        ppp = 1/ff[mm]
        pp.append(ppp)
        
    ## define colormap
    cmap = plt.cm.get_cmap('viridis')
    cmap.set_bad(color='white')
#     cmap.set_under(color='black')
#     cmap.set_over(color='white')       

    ## compute overall maxima and minima
    max_psds = max(nanmax(psds[0]), nanmax(psds[1]), nanmax(psds[2]))
    min_psds = min(nanmin(psds[0]), nanmin(psds[1]), nanmin(psds[2]))
    
    
    ##____________________________
    
#     NN = 3
    N = int(24*365)
    
    font = 14

    fig = plt.figure(constrained_layout=False, figsize=(15,10))
    widths = [6, 1]
    heights = [1, 1, 1]
    spec = fig.add_gridspec(ncols=2, nrows=3, width_ratios=widths, height_ratios=heights)

    plt.subplots_adjust(hspace=0.1, wspace=0.02)

    ax1_1 = fig.add_subplot(spec[0, 0])
    ax1_2 = fig.add_subplot(spec[0, 1], sharey=ax1_1)
    ax2_1 = fig.add_subplot(spec[1, 0], sharex=ax1_1)
    ax2_2 = fig.add_subplot(spec[1, 1], sharey=ax2_1)
    ax3_1 = fig.add_subplot(spec[2, 0], sharex=ax1_1)
    ax3_2 = fig.add_subplot(spec[2, 1], sharey=ax3_1)


    im1 = ax1_1.pcolormesh( times[0], 1/ff[0], psds[0].T, 
                            cmap=cmap,
#                             vmax=max_psds*1e-2,
#                             vmin=min_psds*1e1,
                            norm=colors.LogNorm(vmin, vmax),
                            )
    im2 = ax2_1.pcolormesh( times[1], 1/ff[1], psds[1].T, 
                            cmap=cmap,
#                             vmax=max_psds*1e-2,
#                             vmin=min_psds*1e1,
                            norm=colors.LogNorm(vmin,vmax),
                            )
    im3 = ax3_1.pcolormesh( times[2], 1/ff[2], psds[2].T, 
                            cmap=cmap,
#                             vmax=max_psds*1e-2,
#                             vmin=min_psds*1e1,
                            norm=colors.LogNorm(vmin,vmax),
                            )
    
    set_color = "darkblue"
    
    perc_lower, perc_upper = __get_percentiles((psds[0]))
    ax1_2.fill_betweenx(pp[0], perc_lower, perc_upper, color=set_color, zorder=3, alpha=0.4, label="")
    ax1_2.plot(__get_median_psd(psds[0]), pp[0], color=set_color, zorder=3, alpha=0.9, label="Median")
#     ax1_2.plot(rlnm['rlnm_psd_median'], rlnm['period'], color="black", zorder=2, ls="--", lw=2)
    
    perc_lower, perc_upper = __get_percentiles((psds[1]))
    ax2_2.fill_betweenx(pp[1], perc_lower, perc_upper, color=set_color, zorder=3, alpha=0.4, label="")
    ax2_2.plot(__get_median_psd(psds[1]), pp[1], color=set_color, zorder=3, alpha=0.9, label="Median")
#     ax2_2.plot(rlnm['rlnm_psd_median'], rlnm['period'], color="black", zorder=2, ls="--", lw=2)

    perc_lower, perc_upper = __get_percentiles((psds[2]))
    ax3_2.fill_betweenx(pp[2], perc_lower, perc_upper, color=set_color, zorder=3, alpha=0.4, label="")
    ax3_2.plot(__get_median_psd(psds[2]), pp[2], color=set_color, zorder=3, alpha=0.9, label="Median")
#     ax3_2.plot(rlnm['rlnm_psd_median'], rlnm['period'], color="black", zorder=2, ls="--", lw=2)
        
    
    plt.setp(ax1_1.get_xticklabels(), visible=False)
    plt.setp(ax2_1.get_xticklabels(), visible=False) 
    
    plt.setp(ax1_2.get_xticklabels(), visible=False)
    plt.setp(ax2_2.get_xticklabels(), visible=False) 
    
    plt.setp(ax1_2.get_yticklabels(), visible=False)
    plt.setp(ax2_2.get_yticklabels(), visible=False)    
    plt.setp(ax3_2.get_yticklabels(), visible=False)
    
    
    
        
    for ax in [ax1_1, ax1_2, ax2_1, ax2_2, ax3_1, ax3_2]:
        ax.tick_params(labelsize=font-2)
        ax.set_ylim(config['period_limits'][0], config['period_limits'][1])
        ax.set_xlim(vmin, vmax)
        ax.set_yscale("log")
    
    for ax in [ax1_2, ax2_2, ax3_2]:
        ax.set_xscale("logit")

    ax3_1.set_xlabel("Days of 2019", fontsize=font, labelpad=1)
    ax3_2.set_xlabel(r"PSD (rad$^2$/s$^2$/$Hz$)", fontsize=font, labelpad=-1)

    new_ticks = [int(round(t/24,0)) for t in ax3_1.get_xticks()]
    ax3_1.set_xticklabels(new_ticks)
    
#     ## panel labels
    ax1_1.text(.01, .95, 'a)', ha='left', va='top', color='black', transform=ax1_1.transAxes, fontsize=font+2, 
               bbox=dict(facecolor='white', edgecolor='white', boxstyle='round,pad=0.2'))
    ax2_1.text(.01, .95, 'b)', ha='left', va='top', color='black', transform=ax2_1.transAxes, fontsize=font+2, 
               bbox=dict(facecolor='white', edgecolor='white', boxstyle='round,pad=0.2'))
    ax3_1.text(.01, .95, 'c)', ha='left', va='top', color='black', transform=ax3_1.transAxes, fontsize=font+2, 
               bbox=dict(facecolor='white', edgecolor='white', boxstyle='round,pad=0.2'))
    
#     ## data labels
    array = name.split("_")[0]
    ax1_1.text(.99, .95, f'{array} $vertical$', ha='right', va='top', color='black', backgroundcolor="white", transform=ax1_1.transAxes, fontsize=font,
               bbox=dict(facecolor='white', edgecolor='white', boxstyle='round,pad=0.2'))
    ax2_1.text(.99, .95, f'{array} $north$', ha='right', va='top', color='black', backgroundcolor="white", transform=ax2_1.transAxes, fontsize=font, 
               bbox=dict(facecolor='white', edgecolor='white', boxstyle='round,pad=0.2'))
    ax3_1.text(.99, .95, f'{array} $east$', ha='right', va='top', color='black', backgroundcolor="white", transform=ax3_1.transAxes, fontsize=font, 
               bbox=dict(facecolor='white', edgecolor='white', boxstyle='round,pad=0.2'))
     
    ax1_1.set_ylabel(r"Periods (s)", fontsize=font)
    ax2_1.set_ylabel(r"Periods (s)", fontsize=font)
    ax3_1.set_ylabel(r"Periods (s)", fontsize=font)
    
#     ## set colorbar at bottom 
    cbar = fig.colorbar(im1, orientation='vertical', ax=[ax1_2, ax2_2, ax3_2], aspect=35, pad=0.05)
    cbar.set_label(r"PSD (rad$^2$/s$^2$/$Hz$)", fontsize=font-2, labelpad=-50)

    __savefig(fig, outpath=config['outpath_figures'], outname=f"{name}_psdimage2_test", mode="png", dpi=300)
    
    plt.show();    
#     return fig

In [ ]:
nn = 24*10
__makeplot_image_overview2([ff_Z, ff_N, ff_E], 
                           [ADR_Z[:nn,:], ADR_N[:nn,:], ADR_E[:nn,:]], 
                           [times_Z[:nn], times_N[:nn], times_E[:nn]],
                          )

# __makeplot_image_overview2(
#                                 [ff_Z, ff_N, ff_E], 
#                                 [ADR_Z, ADR_N, ADR_E], 
#                                 [times_Z, times_N, times_E], 
#                                 )

In [ ]:
# __savefig(fig, outpath=config['outpath_figures'], outname=f"{name}_psdimage2_test", mode="png", dpi=300)

## Limit it by computing medians 

In [ ]:
def __makeplot_colorlines_overview(ff, psds, dates=None):
    
    from tqdm.notebook import tqdm
    from numpy import isnan, median, mean, std, array, zeros
    from scipy.stats import median_abs_deviation as mad
    
#     psds_median = __get_median_psd(array(psds))
#     psds_minimal = __get_minimal_psd(array(psds))
#     psds_minimum = __get_minimum_psd(array(psds), ff)

    ## theoretical rlnm
    rlnm = read_csv(config['rlnm_model_path']+"rlnm_theory.csv")

    ## convert frequencies to periods
    pp=[]
    for mm in range(len(ff)):
        ppp = zeros(len(ff[mm]))
        ppp[:-1] = 1/ff[mm][1:]
        pp.append(ppp)
        
    ##____________________________
    
    NN = 3
    
    fig, axes = plt.subplots(NN,1, figsize=(10,10), sharey=False, sharex=True)

    plt.subplots_adjust(hspace=0.1)
    
    font = 14

    N = max(psds[0].shape[0], psds[1].shape[0], psds[2].shape[0])
    colors = plt.cm.rainbow(linspace(0, 1, N))

    ## add Frequency Axis
    g = lambda x: 1/x
    ax2 = axes[0].secondary_xaxis("top", functions=(g,g))
    ax2.set_xlabel("  Frequency (Hz)", fontsize=font, labelpad=-5)
    ax2.set_xticklabels(ff[2], fontsize=11)    
    ax2.tick_params(axis='both', labelsize=font-2)
    
    
    for j in range(NN):
        for n, psd in enumerate(tqdm(psds[j])):
            axes[j].loglog(pp[j], psd, color=colors[n], alpha=0.7)
            p2 = axes[j].scatter(pp[j][0], psd[0], s=0., c=n, cmap='rainbow', vmin=0, vmax=N)

        axes[j].loglog(pp[j], __get_minimum_psd(psds[j], ff[j]), 'grey', zorder=3, alpha=0.9, label="Minimum")
        axes[j].loglog(pp[j], __get_median_psd(psds[j]), 'black', zorder=3, alpha=0.9, label="Median")
        axes[j].loglog(rlnm['period'], rlnm['rlnm_psd_median'], color="black", zorder=2, ls="--", lw=2)
        
        axes[j].grid(True, which="both", ls="-", alpha=0.5)
        axes[j].legend(loc='lower left')
        axes[j].tick_params(labelsize=font-2)
        
        axes[j].text(6, 5e-25, "RLNM", fontsize=font, rotation=-30)

        axes[j].set_xlim(config['period_limits'][0],config['period_limits'][1])
#         axes[j].set_ylim(8e-27, 1e-17)


    axes[NN-1].set_xlabel("  Period (s)", fontsize=font, labelpad=-8)

    ## panel labels
    axes[0].text(.01, .99, 'a)', ha='left', va='top', transform=axes[0].transAxes, fontsize=font+2)
    axes[1].text(.01, .99, 'b)', ha='left', va='top', transform=axes[1].transAxes, fontsize=font+2)
    axes[2].text(.01, .99, 'c)', ha='left', va='top', transform=axes[2].transAxes, fontsize=font+2)
    
    
    axes[0].set_ylabel(r"PSD$_{vertical}$ (rad$^2$/s$^2$/$Hz)$", fontsize=font)
    axes[1].set_ylabel(r"PSD$_{north}$ (rad$^2$/s$^2$/$Hz)$", fontsize=font)
    axes[2].set_ylabel(r"PSD$_{east}$ (rad$^2$/s$^2$/$Hz)$", fontsize=font)
    
    ## set colorbar at bottom
    cbar = fig.colorbar(p2, orientation='vertical', ax=axes.ravel().tolist(), aspect=50, pad=-1e-5)

    
    ## redefine ticks of colorbar
    if dates is not None:
        new_ticks = []
        for t in cbar.get_ticks(): 
            new_ticks.append(dates.astype(str)[int(t)])
        cbar.ax.set_yticklabels(new_ticks)

    plt.show();    
    return fig

In [ ]:
def __compute_medians(arr, intervall=100):

    delta_i = int(intervall)
    d1, d2 = 0, delta_i

    meds = zeros((int(len(arr)/delta_i), shape(arr)[1]))
    idx = 0
    while d2 < len(arr):
        meds[idx] = __get_median_psd(arr[d1:d2,:])
        idx +=1 
        d1 += delta_i
        d2 += delta_i

    return meds

In [ ]:
meds_N = __compute_medians(ADR_N, intervall=24)
meds_E = __compute_medians(ADR_E, intervall=24)
meds_Z = __compute_medians(ADR_Z, intervall=24)

In [ ]:
nn = 100
tmp = linspace(0, shape(ADR_Z)[0], shape(ADR_Z)[0])

fig = __makeplot_colorlines_overview([ff_Z, ff_N, ff_E], [meds_Z, meds_N, meds_E], dates=tmp)

In [ ]:
__savefig(fig, outpath=config['outpath_figures'], outname=f"{name}_colorlines_selection", mode="png", dpi=300)

In [ ]:
# def __makeplot_colorlines(config, ff, psds, dates):

# #     from numpy import isnan, median, mean, std, array
        
#     psds_median  = __get_median_psd(array(psds))
#     psds_minimal = __get_minimal_psd(array(psds))
# #     psds_minimum = __get_minimum_psd(array(psds), ff)
    
        
#     ##____________________________
    
#     fig, ax = plt.subplots(1,1, figsize=(15,10), sharey=False, sharex=True)

#     font = 12

#     N = psds.shape[0]
#     colors = plt.cm.rainbow(linspace(0, 1, N))

#     for n, psd in enumerate(psds):
#         ax.loglog(ff, psd, color=colors[n], alpha=0.7)


#     ## add scatter for colorbar object only
#     for n, psd in enumerate(psds):
#         p2 = ax.scatter(ff[0], psd[0], s=0., c=n, cmap='rainbow', vmin=0, vmax=N)

#     ## plot statistics
#     ax.loglog(ff, psds_median, 'black', zorder=3, alpha=0.9)
# #     ax.loglog(ff, psds_minimal, 'black', zorder=2, alpha=1.0)
# #     ax.loglog(ff, psds_minimum, 'k', zorder=2, alpha=0.8)
    
    
#     ## theoretical rlnm
#     rlnm = read_csv(config['rlnm_model_path']+"rlnm_theory.csv")
#     ax.loglog(rlnm['period'], rlnm['rlnm_psd_median'], color="grey", zorder=2, lw=2, label="RLNM")
    

# #     ax.set_title("Minimal PSD", fontsize=font)
# #     ax.set_title("Minimum PSD", fontsize=font)


# #     ax.set_xlabel("Frequency (Hz)", fontsize=font)
#     ax.set_xlabel("Period (s)", fontsize=font)

#     ax.set_ylabel(r"PSD (rad$^2$/s$^2$/$Hz)$", fontsize=font)
    
#     ax.grid(True, which="both", ls="-", alpha=0.7)
    
#     ax.set_xlim(1,200)
#     ax.set_ylim(1e-26,1e-20)
    
#     ## set colorbar at bottom
#     cbar = fig.colorbar(p2, orientation='horizontal', ax=ax, aspect=50)

#     ## redefine ticks of colorbar
#     new_ticks=[]
#     for t in cbar.get_ticks(): 
#         new_ticks.append(dates.astype(str)[int(t)])
#     cbar.ax.set_xticklabels(new_ticks)


#     plt.show();
    
#     return fig

In [ ]:
# fig, out = __makeplot_colorlines(config, 1/ff, GRF_Z, GRF_Z_df.columns)
# fig, out = __makeplot_colorlines(config, 1/ff, GRF_N, GRF_N_df.columns)
# fig, out = __makeplot_colorlines(config, 1/ff, GRF_E, GRF_E_df.columns)

In [ ]:
plt.pcolormesh(times_Z[:100], ff_Z, ADR_Z[:100,:].T)

In [ ]:
shape(ADR_Z), shape(ff_Z), shape(times_Z)